In [ ]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#BERT

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-multilingual-cased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from datasets import load_dataset
from transformers import BertTokenizer


dataset = load_dataset('text', data_files={'train': '/content/train_lm (1) (1).txt'})



Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=64)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/317842 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-mlm-output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=50,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
)


<ipython-input-6-0d79489e925e>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-mlm-output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=60,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
)

<ipython-input-8-312e6fdde3e9>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
PYTORCH_CUDA_ALLOC_CONF=expandable_segments

NameError: name 'expandable_segments' is not defined

In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: archisman2002 (archisman2002-oracle) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,4.115600
200,3.726200
300,3.511000


Step,Training Loss
100,4.115600
200,3.726200
300,3.511000
400,3.443600
500,3.337600
600,3.279300
700,3.230700
800,3.170200
900,3.176000
1000,3.125300


TrainOutput(global_step=1324, training_loss=3.3384010453238586, metrics={'train_runtime': 2602.735, 'train_samples_per_second': 122.118, 'train_steps_per_second': 0.509, 'total_flos': 1.0465357980672e+16, 'train_loss': 3.3384010453238586, 'epoch': 0.9996224990562477})

In [ ]:
input_file = "/content/test_lm.txt"
output_file = "/content/test_lm.txt"
lines_to_remove = 180000


with open(input_file, "r") as f:
    all_lines = f.readlines()


lines_to_keep = all_lines[:-lines_to_remove] if len(all_lines) > lines_to_remove else []


with open(output_file, "w") as f:
    f.writelines(lines_to_keep)

print(f"Trimmed file saved as {output_file} with {len(lines_to_keep)} lines.")


Trimmed file saved as /content/test_lm.txt with 56554 lines.


In [ ]:

val_dataset = load_dataset('text', data_files={'validation': '/content/test_lm.txt'})


val_tokenized = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])


eval_results = trainer.evaluate(eval_dataset=val_tokenized["validation"])
print(eval_results)


Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/56554 [00:00<?, ? examples/s]

{'eval_loss': nan, 'eval_runtime': 282.0198, 'eval_samples_per_second': 200.532, 'eval_steps_per_second': 25.069, 'epoch': 0.9996224990562477}


In [ ]:
import json
import pandas as pd


with open('final_test.json', 'r', encoding='utf-8') as f:
    texts_data = json.load(f)


labels_df = pd.read_csv('test_labels_hinglish.txt')


uid_to_label = dict(zip(labels_df['Uid'], labels_df['Sentiment']))


texts = []
labels = []

for item in texts_data:
    uid = item['uid']
    text = item['clean_text'].strip()

    if uid in uid_to_label:
        sentiment = uid_to_label[uid].lower()


        if sentiment == 'positive':
            label = 1
        elif sentiment == 'negative':
            label = 0
        elif sentiment == 'neutral':
            label = 2
        else:
            continue

        texts.append(text)
        labels.append(label)

print(f"Loaded {len(texts)} texts for evaluation.")


Loaded 3000 texts for evaluation.


In [ ]:

from transformers import AutoTokenizer

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)


test_encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import DataLoader

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = TestDataset(test_encodings, labels)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [ ]:
from transformers import AutoModelForSequenceClassification

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


model = AutoModelForSequenceClassification.from_pretrained('/content/bert-mlm-output/checkpoint-1324')
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/bert-mlm-output/checkpoint-1324 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels_batch = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        preds = torch.argmax(logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels_batch.cpu().numpy())


accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.64736
Precision: 0.64677
Recall: 0.64892
F1 Score: 0.64087


#DistilBert

In [ ]:

from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("distilbert/distilbert-base-cased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizer


dataset = load_dataset('text', data_files={'train': '/content/train_lm (1) (1).txt'})



Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=64)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/317842 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./distilbert-mlm-output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=50,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
)


<ipython-input-7-506ba18988bf>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./distilbert-mlm-output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=60,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
)

<ipython-input-9-0599d1276cde>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: archisman2002 (archisman2002-oracle) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,3.702000
200,3.358800
300,3.167400
400,3.125700
500,3.006400


Step,Training Loss
100,3.702000
200,3.358800
300,3.167400
400,3.125700
500,3.006400
600,2.963700
700,2.957100
800,2.903100
900,2.895500
1000,2.869000


TrainOutput(global_step=1324, training_loss=3.035565281202426, metrics={'train_runtime': 881.9186, 'train_samples_per_second': 360.398, 'train_steps_per_second': 1.501, 'total_flos': 5265142917857280.0, 'train_loss': 3.035565281202426, 'epoch': 0.9996224990562477})

In [ ]:
input_file = "/content/test_lm.txt"
output_file = "/content/test_lm.txt"
lines_to_remove = 180000


with open(input_file, "r") as f:
    all_lines = f.readlines()


lines_to_keep = all_lines[:-lines_to_remove] if len(all_lines) > lines_to_remove else []


with open(output_file, "w") as f:
    f.writelines(lines_to_keep)

print(f"Trimmed file saved as {output_file} with {len(lines_to_keep)} lines.")


Trimmed file saved as /content/test_lm.txt with 56554 lines.


In [ ]:

val_dataset = load_dataset('text', data_files={'validation': '/content/test_lm.txt'})


val_tokenized = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])


eval_results = trainer.evaluate(eval_dataset=val_tokenized["validation"])
print(eval_results)


Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/56554 [00:00<?, ? examples/s]

{'eval_loss': nan, 'eval_runtime': 106.503, 'eval_samples_per_second': 531.009, 'eval_steps_per_second': 66.383, 'epoch': 0.9996224990562477}


In [ ]:
import json
import pandas as pd


with open('final_test.json', 'r', encoding='utf-8') as f:
    texts_data = json.load(f)


labels_df = pd.read_csv('test_labels_hinglish.txt')


uid_to_label = dict(zip(labels_df['Uid'], labels_df['Sentiment']))


texts = []
labels = []

for item in texts_data:
    uid = item['uid']
    text = item['clean_text'].strip()

    if uid in uid_to_label:
        sentiment = uid_to_label[uid].lower()


        if sentiment == 'positive':
            label = 1
        elif sentiment == 'negative':
            label = 0
        elif sentiment == 'neutral':
            label = 2
        else:
            continue

        texts.append(text)
        labels.append(label)

print(f"Loaded {len(texts)} texts for evaluation.")


Loaded 3000 texts for evaluation.


In [ ]:

from transformers import AutoTokenizer

model_name = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)


test_encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import DataLoader

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = TestDataset(test_encodings, labels)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [ ]:
from transformers import AutoModelForSequenceClassification

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


model = AutoModelForSequenceClassification.from_pretrained('/content/distilbert-mlm-output/checkpoint-1324')
model.to(device)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /content/distilbert-mlm-output/checkpoint-1324 and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels_batch = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        preds = torch.argmax(logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels_batch.cpu().numpy())


accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.63422
Precision: 0.6303
Recall: 0.63219
F1 Score: 0.63505


#RoBERTa

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("aditeyabaral/roberta-hinglish-small")
model = AutoModelForMaskedLM.from_pretrained("aditeyabaral/roberta-hinglish-small")

tokenizer_config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/479k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/279k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/818k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizer


dataset = load_dataset('text', data_files={'train': '/content/train_lm (1) (1).txt'})



In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=64)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/317842 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta-mlm-output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=50,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
)


<ipython-input-29-46e8b64a9e33>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta-mlm-output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=60,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
)

<ipython-input-31-ee515a47c382>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


Step,Training Loss
100,7.126700
200,5.652300
300,5.373700
400,5.175400
500,4.936700
600,4.739500
700,4.597100
800,4.490100
900,4.416400
1000,4.379700


TrainOutput(global_step=1324, training_loss=4.901570397921559, metrics={'train_runtime': 841.1074, 'train_samples_per_second': 377.885, 'train_steps_per_second': 1.574, 'total_flos': 5265329120133120.0, 'train_loss': 4.901570397921559, 'epoch': 0.9996224990562477})

In [ ]:
input_file = "/content/test_lm.txt"
output_file = "/content/test_lm.txt"
lines_to_remove = 180000


with open(input_file, "r") as f:
    all_lines = f.readlines()
print(f"Total lines in file: {len(all_lines)}")


lines_to_keep = all_lines[:-lines_to_remove] if len(all_lines) > lines_to_remove else []


with open(output_file, "w") as f:
    f.writelines(lines_to_keep)

print(f"Trimmed file saved as {output_file} with {len(lines_to_keep)} lines.")


Total lines in file: 236554
Trimmed file saved as /content/test_lm.txt with 136554 lines.


In [ ]:
print(len(dataset["train"]))


317842


In [ ]:

val_dataset = load_dataset('text', data_files={'validation': '/content/test_lm.txt'})


val_tokenized = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])


eval_results = trainer.evaluate(eval_dataset=val_tokenized["validation"])
print(eval_results)


Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/136554 [00:00<?, ? examples/s]

{'eval_loss': nan, 'eval_runtime': 271.717, 'eval_samples_per_second': 502.56, 'eval_steps_per_second': 62.823, 'epoch': 0.9996224990562477}


In [ ]:
import json
import pandas as pd


with open('final_test.json', 'r', encoding='utf-8') as f:
    texts_data = json.load(f)


labels_df = pd.read_csv('test_labels_hinglish.txt')


uid_to_label = dict(zip(labels_df['Uid'], labels_df['Sentiment']))


texts = []
labels = []

for item in texts_data:
    uid = item['uid']
    text = item['clean_text'].strip()

    if uid in uid_to_label:
        sentiment = uid_to_label[uid].lower()


        if sentiment == 'positive':
            label = 1
        elif sentiment == 'negative':
            label = 0
        elif sentiment == 'neutral':
            label = 2
        else:
            continue

        texts.append(text)
        labels.append(label)

print(f"Loaded {len(texts)} texts for evaluation.")


Loaded 3000 texts for evaluation.


In [ ]:

from transformers import AutoTokenizer

model_name = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)


test_encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)


In [ ]:
import torch
from torch.utils.data import DataLoader

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = TestDataset(test_encodings, labels)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [ ]:
from transformers import AutoModelForSequenceClassification

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


model = AutoModelForSequenceClassification.from_pretrained('/content/distilbert-mlm-output/checkpoint-1324')
model.to(device)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /content/distilbert-mlm-output/checkpoint-1324 and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels_batch = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        preds = torch.argmax(logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels_batch.cpu().numpy())


accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.62441
Precision: 0.63179
Recall: 0.63619
F1 Score: 0.62013
